# SEP a analizar 
<div>
<img src="SEP.png" width="500"/>
</div>


## Creación del SEP


### Obteniendo Ybus

In [47]:
import pandapower as pp
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd

import pandapower.plotting as pplot
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


#Crear red vacía
net = pp.create_empty_network()

#Crear barras
bus1 = pp.create_bus(net, vn_kv=110, name="Barra 1")
bus2 = pp.create_bus(net, vn_kv=220, name="Barra 2")
bus1A = pp.create_bus(net, vn_kv=220, name="Barra 1A")
bus2A = pp.create_bus(net, vn_kv=220, name="Barra 2A")
bus3A = pp.create_bus(net, vn_kv=220, name="Barra 3A")
bus1B = pp.create_bus(net, vn_kv=220, name="Barra 1B")
bus2B = pp.create_bus(net, vn_kv=220, name="Barra 2B")

#Crear barra slack en Barra 1
pp.create_ext_grid(net, bus1, vm_pu=1, va_degree=0.0, name="Slack")

#Crear transformador entre Barra 1 y Barra 2
pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, std_type="100 MVA 220/110 kV", name="Trafo 1-2")

#Crear líneas de transmisión
pp.create_line(net, from_bus=bus2, to_bus=bus1A, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2-1A")
pp.create_line(net, from_bus=bus1A, to_bus=bus2A, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 1A-2A")
pp.create_line(net, from_bus=bus2A, to_bus=bus3A, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2A-3A")
pp.create_line(net, from_bus=bus3A, to_bus=bus2B, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 3A-2B")
pp.create_line(net, from_bus=bus2B, to_bus=bus1B, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 1B-2B")
pp.create_line(net, from_bus=bus1B, to_bus=bus2, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2-1B")

#Crear cargas
#pp.create_load(net, bus2, p_mw=150., q_mvar=100., name="Load 2")  # Carga nominal en Barra 2
pp.create_load(net, bus1A, p_mw=30., q_mvar=20., name="Load 1A")
pp.create_load(net, bus2A, p_mw=52.5, q_mvar=35., name="Load 2A")
pp.create_load(net, bus3A, p_mw=22.5, q_mvar=15., name="Load 3A")
pp.create_load(net, bus2B, p_mw=90., q_mvar=60., name="Load 2B")
pp.create_load(net, bus1B, p_mw=15., q_mvar=10., name="Load 1B")


pp.runpp(net)

# Acceder a la matriz Ybus directamente desde el resultado del flujo de carga
Ybus = net._ppc['internal']['Ybus']
#print(Ybus)
# Configurar NumPy para mostrar solo 3 decimales
np.set_printoptions(precision=3, suppress=True)

# Convertir Ybus a un array denso
Ybus_array = Ybus.toarray()  # Convertir de matriz dispersa a densa

# Crear un DataFrame de pandas con la matriz Ybus
Ybus_df = pd.DataFrame(Ybus_array)

# Configurar pandas para mejorar la visualización
pd.set_option('display.max_columns', None)  # Asegura mostrar todas las columnas
pd.set_option('display.width', 1000)  # Ajusta el ancho para la visualización
pd.set_option('display.float_format', '{:.3f}'.format)  # Formato de los números flotantes

# Mostrar el DataFrame
print("Matriz de Admitancia Ybus:")
print(Ybus_df)



Matriz de Admitancia Ybus:
                 0                     1                     2                    3                    4                     5                    6
0  18.069-833.144j      -18.042+833.132j          0.000+0.000j         0.000+0.000j         0.000+0.000j          0.000+0.000j         0.000+0.000j
1 -18.042+833.132j  28090.835-45050.010j -14036.383+22117.937j         0.000+0.000j         0.000+0.000j -14036.383+22117.937j         0.000+0.000j
2     0.000+0.000j -14036.383+22117.937j  23393.971-36839.469j -9357.589+14745.291j         0.000+0.000j          0.000+0.000j         0.000+0.000j
3     0.000+0.000j          0.000+0.000j  -9357.589+14745.291j 16375.780-25770.998j -7018.191+11058.968j          0.000+0.000j         0.000+0.000j
4     0.000+0.000j          0.000+0.000j          0.000+0.000j -7018.191+11058.968j 16375.780-25770.998j          0.000+0.000j -9357.589+14745.291j
5     0.000+0.000j -14036.383+22117.937j          0.000+0.000j         0.000+0.000j  

### Matriz con los ángulos 

In [46]:
# Paso 2: Crear una matriz vacía del mismo tamaño que Ybus_array para los ángulos en grados.
Ybus_angles = np.zeros(Ybus_array.shape)

# Paso 3: Recorrer cada elemento de Ybus_array para calcular los ángulos en grados.
for i in range(Ybus_array.shape[0]):
    for j in range(Ybus_array.shape[1]):
        # Calcular el ángulo en radianes y luego convertir a grados.
        Ybus_angles[i, j] = np.degrees(np.angle(Ybus_array[i, j]))

# Paso 4: Crear un DataFrame de pandas con la matriz de ángulos.
Ybus_angles_df = pd.DataFrame(Ybus_angles)

# Paso 5: Mostrar el DataFrame de ángulos.
print("Matriz de Ángulos de Admitancia Ybus en Grados:")
print(Ybus_angles_df)

Matriz de Ángulos de Admitancia Ybus en Grados:
        0       1       2       3       4       5       6
0 -88.758  91.241   0.000   0.000   0.000   0.000   0.000
1  91.241 -58.054 122.400   0.000   0.000 122.400   0.000
2   0.000 122.400 -57.583 122.400   0.000   0.000   0.000
3   0.000   0.000 122.400 -57.567 122.400   0.000   0.000
4   0.000   0.000   0.000 122.400 -57.567   0.000 122.400
5   0.000 122.400   0.000   0.000   0.000 -57.567 122.400
6   0.000   0.000   0.000   0.000 122.400 122.400 -57.550


## Tabla

In [ ]:

# Crear un DataFrame para almacenar los resultados de las barras
barra_resultados = pd.DataFrame(columns=["Tipo de Barra", "P (MW)", "Q (MVAr)", "V (p.u.)", "Ángulo (°)"])

# Obtener los resultados de las barras
bus_results = net.res_bus

# Definir manualmente los tipos de barras basado en la configuración de la red
tipo_barra_manual = {bus1: "Slack", bus2: "PQ", bus1A: "PQ", bus2A: "PQ", bus3A: "PQ", bus1B: "PQ", bus2B: "PQ"}

# Iterar sobre cada barra y almacenar sus resultados en el DataFrame
rows_list = []
for i, bus in net.bus.iterrows():
    tipo_barra = tipo_barra_manual.get(bus.name, "PQ")  
    p = bus_results.at[i, "p_mw"] if tipo_barra == "PQ" else "N/A"
    q = bus_results.at[i, "q_mvar"] if tipo_barra == "PQ" else "N/A"
    v = bus_results.at[i, "vm_pu"]
    angulo = bus_results.at[i, "va_degree"]
    rows_list.append({"Tipo de Barra": tipo_barra, "P (MW)": p, "Q (MVAr)": q, "V (p.u.)": v, "Ángulo (°)": angulo})

barra_resultados = pd.DataFrame(rows_list)
# Mostrar el DataFrame de resultados
print("Resultados de las Barras:")
print(barra_resultados)


NameError: name 'pd' is not defined

## Newthon


In [52]:
# Paso 1: Obtener las dimensiones de Ybus_array
num_rows, num_cols = Ybus_array.shape

# Paso 2: Crear matrices vacías para las partes real e imaginaria
Ybus_real = np.zeros((num_rows, num_cols))
Ybus_imag = np.zeros((num_rows, num_cols))

# Paso 3: Iterar sobre cada elemento de Ybus_array y separar las partes real e imaginaria
for i in range(num_rows):
    for j in range(num_cols):
        Ybus_real[i, j] = Ybus_array[i, j].real
        Ybus_imag[i, j] = Ybus_array[i, j].imag


# Voltajes iniciales (pu) y ángulos iniciales (radianes)
V = np.array([1, 1, 1, 1, 1, 1, 1], dtype=np.float64)  # Asumiendo 1 pu para todas las barras inicialmente
theta = np.zeros(7, dtype=np.float64)  # Asumiendo 0 radianes inicialmente

# Matriz de admitancia Y_bus (parte real e imaginaria)
Y_real = Ybus_real
Y_imag = Ybus_imag

# Ajuste de los arrays P_inyectado y Q_inyectado para incluir todas las barras PQ (en pu)
base_power = 100e6  # Potencia base en Watts
P_inyectado = np.array([0, 150, 30, 52.5, 22.5, 15, 90]) / base_power  # Convertido a pu
Q_inyectado = np.array([0, 100, 20, 35, 15, 10, 60]) / base_power  # Convertido a pu

#Función para calcular las derivadas parciales H, N, M, L según las definiciones dadas
def calcular_derivadas(V, theta, Y_real, Y_imag):
    n = len(V)
    H = np.zeros((n, n))
    N = np.zeros((n, n))
    M = np.zeros((n, n))
    L = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                H[i, j] = -V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
                N[i, j] = -V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                M[i, j] = V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                L[i, j] = -V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
            else:
                H[i, i] = Y_imag[i, i] * V[i]**2
                N[i, i] = -Y_real[i, i] * V[i]
                M[i, i] = -Y_imag[i, i] * V[i]
                L[i, i] = -Y_real[i, i] * V[i]**2

    return H, N, M, L

#Función para calcular el vector f(x) (delta P y delta Q)
def calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado):
    n = len(V)
    delta_P = np.zeros(n)
    delta_Q = np.zeros(n)

    for i in range(n):
        sum_P = 0.0
        sum_Q = 0.0
        for j in range(n):
            sum_P += V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
            sum_Q += V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))

        delta_P[i] = P_inyectado[i] - sum_P
        delta_Q[i] = Q_inyectado[i] - sum_Q

    f_x = np.concatenate((delta_P, delta_Q))
    return f_x
    #Función para calcular el Jacobiano J según las definiciones dadas
def calcular_jacobiano(V, theta, Y_real, Y_imag):
    n = len(V)
    H, N, M, L = calcular_derivadas(V, theta, Y_real, Y_imag)

    J = np.zeros((2 * n, 2 * n))

    # Matrices H, N, M, L en el Jacobiano J
    J[:n, :n] = H
    J[:n, n:] = N
    J[n:, :n] = M
    J[n:, n:] = L

    return J

#Función para ejecutar el método de Newton-Raphson
def newton_raphson(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado, max_iter=100, tol=1e-6):
    n = len(V)
    iteracion = 0
    delta = np.inf

    while delta > tol and iteracion < max_iter:
        # Calcular el Jacobiano J
        J = calcular_jacobiano(V, theta, Y_real, Y_imag)

        # Calcular el vector f(x) (delta P y delta Q)
        f_x = calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado)
        J_pseudo_inv = np.linalg.pinv(J)
        delta_x = -np.dot(J_pseudo_inv, f_x)
        # Calcular delta_x usando la inversa de J
        #delta_x = -np.linalg.solve(J, f_x)

        # Actualizar V y theta
        V += delta_x[:n]
        theta += delta_x[n:]

        # Calcular la norma del vector f(x) para criterio de convergencia
        delta = np.linalg.norm(f_x)

        # Incrementar contador de iteraciones
        iteracion += 1

    if delta <= tol:
        print(f"Convergencia alcanzada en {iteracion} iteraciones.")
    else:
        print("El método de Newton-Raphson no convergió después de", max_iter, "iteraciones.")

    return V, theta

# Llamar a la función newton_raphson
V_final, theta_final = newton_raphson(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado)

# Imprimir los resultados
print("Voltajes finales (pu):", V_final)
print("Ángulos finales (radianes):", theta_final)

El método de Newton-Raphson no convergió después de 100 iteraciones.
Voltajes finales (pu): [ 17.325 -19.182 -35.977 -45.688 -34.721 -48.166  14.038]
Ángulos finales (radianes): [-4092.359   -96.279   -66.44    -86.48    -92.111   -83.872  -102.627]


In [56]:
def calcular_P_Q(V, theta, Y_real, Y_imag):
    n = len(V)
    P_calculado = np.zeros(n)
    Q_calculado = np.zeros(n)

    for i in range(n):
        for j in range(n):
            P_calculado[i] += V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
            Q_calculado[i] += V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))

    return P_calculado, Q_calculado

# Llamar a la función calcular_P_Q después de ejecutar newton_raphson
P_final, Q_final = calcular_P_Q(V_final, theta_final, Y_real, Y_imag)

# Imprimir los resultados
print("Potencias activas finales (pu):", P_final)
print("Potencias reactivas finales (pu):", Q_final)

Potencias activas finales (pu): [     4119.482  16117844.279  31876253.029 -13530777.406  25822995.539
  30990944.492  -3040030.838]
Potencias reactivas finales (pu): [   526988.606 -14975410.775  34187765.293  51910439.073   3005464.1
  54877556.587  10342519.466]


In [53]:
# Asumiendo que las variables V, theta, Y_real, Y_imag ya están definidas y calculadas como en el código proporcionado

# Calcular las matrices H, N, M, L
H, N, M, L = calcular_derivadas(V, theta, Y_real, Y_imag)

# Imprimir las matrices
print("Matriz H:")
print(H)
print("\nMatriz N:")
print(N)
print("\nMatriz M:")
print(M)
print("\nMatriz L:")
print(L)

Matriz H:
[[  -250061.433   -276927.172         0.            0.            0.
          0.           -0.   ]
 [  -276611.115 -16575930.373   9591004.125        -0.           -0.
   22236948.139         0.   ]
 [        0.     -9781657.151 -47682108.859  23276000.717        -0.
          0.           -0.   ]
 [        0.           -0.     -5290273.057 -53794209.977   7174043.961
         -0.            0.   ]
 [        0.           -0.            0.     20698406.372 -31067684.255
         -0.      7363813.783]
 [       -0.     18113867.339        -0.           -0.           -0.
  -68329961.181  -4661462.745]
 [        0.           -0.            0.            0.      -728298.045
   -5263721.954  -4350499.467]]

Matriz N:
[[   -313.044     -67.974      -0.         -0.         -0.         -0.
       -0.   ]
 [   -767.173  538835.447  425937.188       0.          0.    -198377.716
        0.   ]
 [     -0.    -792560.546  841637.347  367705.99        0.          0.
        0.   ]
 [     -

In [55]:
# Función para imprimir los valores de una matriz con sus posiciones
def imprimir_matriz_con_posiciones(matriz, nombre_matriz):
    num_filas, num_columnas = matriz.shape
    for i in range(num_filas):
        for j in range(num_columnas):
            print(f"{nombre_matriz}({i+1},{j+1}) = {matriz[i, j]}")

# Asumiendo que las matrices H, N, M, L ya están calculadas

# Imprimir los valores de H con sus posiciones
imprimir_matriz_con_posiciones(H, "H")
print("\n")  # Espacio entre matrices

# Imprimir los valores de N con sus posiciones
imprimir_matriz_con_posiciones(N, "N")
print("\n")  # Espacio entre matrices

# Imprimir los valores de M con sus posiciones
imprimir_matriz_con_posiciones(M, "M")
print("\n")  # Espacio entre matrices

# Imprimir los valores de L con sus posiciones
imprimir_matriz_con_posiciones(L, "L")

H(1,1) = -250061.43329085488
H(1,2) = -276927.17248012114
H(1,3) = 0.0
H(1,4) = 0.0
H(1,5) = 0.0
H(1,6) = 0.0
H(1,7) = -0.0
H(2,1) = -276611.1151191996
H(2,2) = -16575930.373301892
H(2,3) = 9591004.124931455
H(2,4) = -0.0
H(2,5) = -0.0
H(2,6) = 22236948.13851529
H(2,7) = 0.0
H(3,1) = 0.0
H(3,2) = -9781657.151475187
H(3,3) = -47682108.85877044
H(3,4) = 23276000.716882292
H(3,5) = -0.0
H(3,6) = 0.0
H(3,7) = -0.0
H(4,1) = 0.0
H(4,2) = -0.0
H(4,3) = -5290273.056763862
H(4,4) = -53794209.977287225
H(4,5) = 7174043.960660974
H(4,6) = -0.0
H(4,7) = 0.0
H(5,1) = 0.0
H(5,2) = -0.0
H(5,3) = 0.0
H(5,4) = 20698406.37173109
H(5,5) = -31067684.255439892
H(5,6) = -0.0
H(5,7) = 7363813.783429578
H(6,1) = -0.0
H(6,2) = 18113867.33852496
H(6,3) = -0.0
H(6,4) = -0.0
H(6,5) = -0.0
H(6,6) = -68329961.18066081
H(6,7) = -4661462.745248037
H(7,1) = 0.0
H(7,2) = -0.0
H(7,3) = 0.0
H(7,4) = 0.0
H(7,5) = -728298.0451497274
H(7,6) = -5263721.954100331
H(7,7) = -4350499.467015329


N(1,1) = -313.04361991334036
N(1,